In [ ]:
#a patch with mainly a single color.
# img[:, X_mid-N_size/2:X_mid+N_size/2, Y_mid-N_size/2:Y_mid+N_size/2]  
# X_mid, Y_mid = 825, 1325 and N_size = 50

crop_a = img[:, 600:650, 800:850]
plt.imshow(rearrange(crop_a, 'c h w -> h w c'))


In [ ]:
#a patch with 2-3 different colors.
# img[:, X_mid-N_size/2:X_mid+N_size/2, Y_mid-N_size/2:Y_mid+N_size/2]  
# X_mid, Y_mid = 850, 650 and N_size = 50

crop_b = img[:, 650:700, 825:875]
plt.imshow(rearrange(crop_b, 'c h w -> h w c'))

In [ ]:
# a patch with at least 5 different colors.
# img[:, X_mid-N_size/2:X_mid+N_size/2, Y_mid-N_size/2:Y_mid+N_size/2]  
# X_mid, Y_mid = 800, 950 and N_size = 50

crop_c = img[:, 775:825, 925:975]
plt.imshow(rearrange(crop_c, 'c h w -> h w c'))

In [ ]:
def plot_reconstructed_img_d(reconstructed_img3D, masked_img, img, X_mid, Y_mid, RMSE, PSNR, title, N_size, r):    
    fig = plt.figure(figsize=(5, 2.5))
    gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1])
    
    ax0 = plt.subplot(gs[0])
    ax1 = plt.subplot(gs[1])

    reconstructed_img = img.clone()
    reconstructed_img[:, X_mid-int(N_size/2):X_mid+int(N_size/2), Y_mid-int(N_size/2):Y_mid+int(N_size/2)] = torch.tensor(reconstructed_img3D)
    ax0.imshow(rearrange(reconstructed_img, 'c h w -> h w c'))
    ax0.set_title(f"Reconstructed Image [r = {r}]")

    ax1.imshow(rearrange(img, 'c h w -> h w c'))
    ax1.set_title(f"original image")

    for a in [ax0, ax1]:
        a.axis("off")

    plt.suptitle(title, weight='bold')
    # fig.suptitle(f"RMSE = {RMSE:.2f}, PSNR = {PSNR:.2f}", weight='bold', y=1.0)
    plt.tight_layout()

In [ ]:
def reconstruct_img_d(img, k, N_size, X_mid, Y_mid, title):
    # output: reconstructed_img_scaled, reconstructed_img3D
    #          for plotting , for metrics
    
    new_img = img[:, int(X_mid-N_size/2):int(X_mid+N_size/2), int(Y_mid-N_size/2):int(Y_mid+N_size/2)]

    # convert masked image into a 2D tensor to factorise
    new_img_2D = new_img.reshape(-1, new_img.size(-1))

    W, H, loss = factorize(new_img_2D, k, device=device)
    reconstructed_img = torch.mm(W, H)

    # convert the reconstructed 2D tensor to 3D tensor
    reconstructed_img3D = reconstructed_img.reshape(3, new_img.shape[1], -1)
    
    # Reshaping and Scaling 
    reconstructed_img_scaled = reshaping(reconstructed_img3D)
    
    # calculate metrics
    RMSE, PSNR = metrics(reconstructed_img3D, new_img)
    
    #plot reconstructed and masked image
    plot_reconstructed_img_d(reconstructed_img_scaled, new_img, img, X_mid, Y_mid, RMSE, PSNR, title, N_size, k)
    plot_reconstructed_img(reconstructed_img_scaled, new_img, RMSE, PSNR, 'Zoomed in Comparision', N_size)
    return new_img, RMSE, PSNR

In [ ]:
def plot_metrics(r_values, RMSE_values, PSNR_values):
    print("r_values:", r_values)
    print("RMSE_values:", RMSE_values)
    print("PSNR_values:", PSNR_values)
    fig, axs = plt.subplots(1, 2, figsize=(5, 2.5))  
    axs[0].plot(r_values, RMSE_values, marker='o', label='RMSE')
    axs[0].set_title('RMSE vs r size')
    axs[0].grid(True)
    axs[0].legend()
    
    axs[1].plot(r_values, PSNR_values, marker='o', label='PSNR')
    axs[1].set_title('PSNR vs r size')
    axs[1].grid(True)
    axs[1].legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
# a.An Area with mainly a Single Color
r = [5,10,25,50]
RMSE_a = []
PSNR_a = []
for i in range(4):
    masked_img_a, RMSE, PSNR = reconstruct_img_d(img, r[i], 50, 625, 825, 'An Area with mainly a Single Color')
    RMSE_a.append(RMSE)
    PSNR_a.append(PSNR)

In [ ]:
plot_metrics(r, RMSE_a, PSNR_a)


In [ ]:
# b.A patch with 2-3 different colors.
r = [5,10,25,50]
RMSE_b = []
PSNR_b = []
for i in range(4):
    masked_img_b, RMSE, PSNR = reconstruct_img_d(img, r[i], 50, 675, 850, 'A Patch with 2-3 Different Colors')
    RMSE_b.append(RMSE)
    PSNR_b.append(PSNR)

In [ ]:
plot_metrics(r, RMSE_b, PSNR_b)

In [ ]:

# c.A Patch with at least 5 Different colors.
r = [5,10,25,50]
RMSE_c = []
PSNR_c = []
for i in range(4):
    masked_img_c, RMSE, PSNR = reconstruct_img_d(img, r[i], 50, 800, 950, 'A Patch with at least 5 different colors')
    RMSE_c.append(RMSE)
    PSNR_c.append(PSNR)

In [ ]:
plot_metrics(r, RMSE_c, PSNR_c)